In [1]:
from requests import request
from requests.compat import *
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
import pandas as pd
import math

def makePageDF(category_id, page_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:80]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    total_page = math.ceil(total/80)
    
    df = makePageDF(category_id,1)
    
    for j in range(1,10):
        if total_page >1:
            for i in range(2,total_page+1):
                new_df = makePageDF(category_id,i)
                df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [2]:
df, merged_df, attributes = makeCategoryDF(100001011)

KeyboardInterrupt: 

In [ ]:
df

In [ ]:
merged_df

In [ ]:
attributes

In [ ]:
def makePageDF(category_id, page_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:80]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    total_page = math.ceil(total/80)
    
    df = makePageDF(category_id,1)
    
    for j in range(1,5):
        if total_page >1:
            for i in range(2,total_page+1):
                new_df = makePageDF(category_id,i)
                df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [ ]:
df, merged_df, attributes = makeCategoryDF(100001011)

In [ ]:
df

In [ ]:
def makePageDF(category_id, page_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : '40',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:40]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '40',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    total_page = math.ceil(total/40)
    
    df = makePageDF(category_id,1)
    
    if total_page >1:
        for i in range(2,total_page+1):
            new_df = makePageDF(category_id,i)
            df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [ ]:
df, merged_df, attributes = makeCategoryDF(100001011)

In [ ]:
df

In [ ]:
def makePageDF(category_id, page_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:80]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    total_page = math.ceil(total/80)
    
    df = makePageDF(category_id,1)
    

    if total_page >1:
        for i in range(2,total_page+1):
            new_df = makePageDF(category_id,i)
            df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [ ]:
df, merged_df, attributes = makeCategoryDF(100001011)

In [ ]:
df

In [ ]:
def makePageDF(category_id, page_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : '60',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:60]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '60',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    total_page = math.ceil(total/60)
    
    df = makePageDF(category_id,1)
    
    if total_page >1:
        for i in range(2,total_page+1):
            new_df = makePageDF(category_id,i)
            df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [ ]:
df, merged_df, attributes = makeCategoryDF(100001011)

In [ ]:
df

In [ ]:
def makePageDF(category_id, page_id, pagingsize):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : str(pagingsize),
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:pagingsize]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    
    df = pd.DataFrame()
    
    for j in [20,40,60,80]:
        total_page = math.ceil(total/j)
        for i in range(1,total_page+1):
            new_df = makePageDF(category_id,i,j)
            df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [ ]:
df, merged_df, attributes = makeCategoryDF(100001011)

In [ ]:
df

In [ ]:
def makePageDF(category_id, page_id, pagingsize):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : str(pagingsize),
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    print(resp.request.url)
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:pagingsize]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    
    df = pd.DataFrame()
    
    for j in [60,80]:
        total_page = math.ceil(total/j)
        for i in range(1,total_page+1):
            new_df = makePageDF(category_id,i,j)
            df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [ ]:
df, merged_df, attributes = makeCategoryDF(100001011)

In [ ]:
df